#  Rounding off to nearest even and odd integers

In [1]:
function round_odd(N)
   val=Int64(round(N))
    if (val%2)==0     
     if (N-val)>=0
        val+=1
     else
        val-=1
    end
    end
   return val     
end

function round_even(N)
   val=Int64(round(N))
    if (val%2)==1    
     if (N-val)>0
        val+=1
     else
        val-=1 
     end
    end
   return val     
end

round_even (generic function with 1 method)

# Script for calculating Symmetry expectations 

### The following function returns $\langle Z_{1}X_{2}X_{5}...X_{N-1}Z_{N}\rangle$ and $\langle X_{1}X_{3}X_{5}...X_{N}\rangle$ with respect to the MPS representation T. 

In [6]:
using ITensors


function Sym1(T::MPS,N) 
   
   mps=copy(T);
   val=copy(T);
    
   s = siteind(mps,1)
   newpsi= 2*op(s,"Sz")*mps[1]
   noprime!(newpsi)
   mps[1]= newpsi
    
    for o = (2):2:(N-1)
           s = siteind(mps,o)
           newpsi= 2*op(s,"Sx")*mps[o]
           noprime!(newpsi)
           mps[o]= newpsi
    end
    
   s = siteind(mps,N)
   newpsi= 2*op(s,"Sz")*mps[N]
   noprime!(newpsi)
   mps[N]= newpsi
    
   expectation=inner(mps,val)
    
  return expectation
end

function Sym2(T::MPS,N)  
   
   mps=copy(T);
   val=copy(T);
    
    for o = (1):2:(N)
           s = siteind(mps,o)
           newpsi= 2*op(s,"Sx")*mps[o]
           noprime!(newpsi)
           mps[o]= newpsi
    end
    
    
   expectation=inner(mps,val)
    
  return expectation
end

Sym2 (generic function with 1 method)

In [8]:
using ITensors


function U1(T::MPS,N) 
   
   mps=copy(T);
   val=copy(T);
    
   s = siteind(mps,1)
   newpsi= 2*op(s,"Sz")*mps[1]
   noprime!(newpsi)
   mps[1]= newpsi
    
    for o = (2):2:(N-1)
           s = siteind(mps,o)
           newpsi= 2*op(s,"Sx")*mps[o]
           noprime!(newpsi)
           mps[o]= newpsi
    end
    
   s = siteind(mps,N)
   newpsi= 2*op(s,"Sz")*mps[N]
   noprime!(newpsi)
   mps[N]= newpsi
    
    
  return mps
end

function U2(T::MPS,N)  
   
   mps=copy(T);
   val=copy(T);
    
    for o = (1):2:(N)
           s = siteind(mps,o)
           newpsi= 2*op(s,"Sx")*mps[o]
           noprime!(newpsi)
           mps[o]= newpsi
    end
    
    
  return mps
end


U2 (generic function with 1 method)

In [7]:



using ITensors

function R1(T::MPS,start,finish)
    
    if start==finish
        return 1
    else
   
       mps=copy(T)

       s = siteind(mps,start)
       newpsi= 2*op(s,"Sz")*mps[start]
       noprime!(newpsi)
       mps[start]= newpsi

        for o = (start+1):2:(finish-1)
               s = siteind(mps,o)
               newpsi= 2*op(s,"Sx")*mps[o]
               noprime!(newpsi)
               mps[o]= newpsi
        end

       s = siteind(mps,finish)
       newpsi= 2*op(s,"Sz")*mps[finish]
       noprime!(newpsi)
       mps[finish]= newpsi


        return mps 
    end
end


function R2(T::MPS,start,finish)
    
    if start==finish
        return 1
    else
   
       mps=copy(T)

       s = siteind(mps,start)
       newpsi= 2*op(s,"Sz")*mps[start]
       noprime!(newpsi)
       mps[start]= newpsi

        for o = (start+1):2:(finish)
               s = siteind(mps,o)
               newpsi= 2*op(s,"Sx")*mps[o]
               noprime!(newpsi)
               mps[o]= newpsi
        end


        return mps 
    end
end

R2 (generic function with 1 method)

In [10]:

using ITensors
    acc=1E-13
    bond_dim=50
    Ndel=100
    Nlist=[401]
    SOP1= Array{Float64,1}(undef,Ndel+1)
    SOP2= Array{Float64,1}(undef,Ndel+1)
    error=Array{Float64,1}(undef,Ndel+1)
for N in Nlist # only odd input please
    sites = siteinds("S=1/2",N)
    psi0 = randomMPS(sites,bond_dim)
    @show Initial=round_odd(N/2) 
    printstyled("START OF A NEW CHAIN LENGTH \n"; color = :red)
    println("Chain length=$N")
    
    for nloop in 1:(Ndel+1)
    
      alpha= pi*(nloop-1)/(2*Ndel)
      s_p=((nloop-1)*100/Ndel);
      cs=cos(alpha)
      ss=sin(alpha)
      ampo = AutoMPO()
      for j=2:N-1
        ampo += -8*cs,"Sz",j-1,"Sx",j,"Sz",j+1;
      end
      ampo+= -4*cs,"Sz",N-1,"Sx",N;
      ampo+= -4*cs,"Sx",1,"Sz",2;
      for j=2:N-1
        ampo+= -2*ss,"Sx",j
      end 
      H = MPO(ampo,sites)
      println("$s_p % into the sweep")
      sweeps = Sweeps(15) # number of sweeps is 15
      maxdim!(sweeps,10,20,50,100,200,400,500,600,700,800,1000) # gradually increase states kept
      cutoff!(sweeps,acc) # desired truncation error
      energy,T = dmrg(H,psi0,sweeps,outputlevel=0; svd_alg="qr_iteration")#Run the dmrg algorithm
      error[nloop]= ((inner(H,T,H,T)-energy*energy)/N^2)
      phi_1=U1(T,N)
      phi_2=U2(T,N)
      phi_12=U1(phi_2,N)
      norm= 1+inner(phi_1,T)+inner(phi_2,T)+inner(phi_12,T)
      @show norm 
      R1_T= R1(T,Initial,N)
      R2_T= R2(T,Initial+1,N)
      SOP1[nloop]=(inner(R1_T,T)+inner(R1_T,phi_1)+inner(R1_T,phi_2)+inner(R1_T,phi_12))/norm;
      SOP2[nloop]=(inner(R2_T,T)+inner(R2_T,phi_1)+inner(R2_T,phi_2)+inner(R2_T,phi_12))/norm;
      @show SOP1[nloop]
      @show SOP2[nloop]
       
    end
   using DelimitedFiles
   writedlm("Sym_subspace SOP1 N=$N accuracy=$acc.csv",SOP1,',')
   writedlm("Sym_subspace SOP2 N=$N accuracy=$acc.csv",SOP2,',')
   writedlm("Sym_subspace Error N=$N accuracy=$acc.csv",error,',')
end


Initial = round_odd(N / 2) = 201
START OF A NEW CHAIN LENGTH 
Chain length=401
0.0 % into the sweep
norm = 3.999999999999985
SOP1[nloop] = 0.9999999999999988
SOP2[nloop] = 0.9999999999999998
1.0 % into the sweep
norm = 4.0
SOP1[nloop] = 0.9999691490811251
SOP2[nloop] = 0.9999691490811254
2.0 % into the sweep
norm = 4.000000000000019
SOP1[nloop] = 0.9998764953516157
SOP2[nloop] = 0.9998764953516159
3.0 % into the sweep
norm = 4.0000000000000195
SOP1[nloop] = 0.9997217350548797
SOP2[nloop] = 0.9997217350548815
4.0 % into the sweep
norm = 4.000000000000013
SOP1[nloop] = 0.9995043591231418
SOP2[nloop] = 0.9995043591231415
5.0 % into the sweep
norm = 4.000000000000031
SOP1[nloop] = 0.9992236489189634
SOP2[nloop] = 0.9992236489189628
6.0 % into the sweep
norm = 4.000000000000007
SOP1[nloop] = 0.9988786701753741
SOP2[nloop] = 0.998878670175373
7.0 % into the sweep
norm = 3.9999999999818385
SOP1[nloop] = 0.9984682650121545
SOP2[nloop] = 0.9984682650155888
8.0 % into the sweep
norm = 3.99999999

In [ ]:
using LaTeXStrings
N =7 # only odd input please
acc=1E-13
Ndel=100
using DelimitedFiles
SOP1= readdlm("Sym_subspace SOP1 N=$N accuracy=$acc.csv",',')
SOP2= readdlm("Sym_subspace SOP2 N=$N accuracy=$acc.csv",',')
swe= LinRange(0,1,Ndel+1)

using Plots
pyplot()
plt=plot!(swe,SOP1, #ylims = (0,1),
    title ="Symmetry Check",label=:false,linecolor=:red,
    legend=:topright,fmt = :png,dpi=300,reuse=false) 
xlabel!(L"\frac{2\alpha}{\pi}")
ylabel!(L"\langle g_{01} \rangle")
scatter!(swe,Symmetry[:,1],label="N=$N",markersize=4,markercolor=:red,reuse=false,markershape=:circle)
#display(plt2)
#png("2ν₀₁ diff chain lengths accuracy=$acc")



In [34]:
using ITensors

  N = 10
  sites = siteinds("Qubit",N;conserve_qns=true)
  state_a = [ (isequal(n,1)|| isequal(n,N)) ? "0" : "X+" for n=1:N ] 
  psi_a = productMPS(sites,state)


LoadError: ArgumentError: Overload of "state" or "state!" functions not found for state name "X+" and Index tags "Qubit,Site,n=1"